In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import string 
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from sklearn.utils import shuffle

In [ ]:
#downloading the wordnet
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# reading the dataset into a dataframe
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
df = pd.read_csv('Twitter1.csv',encoding = "ISO-8859-1",names=DATASET_COLUMNS)

In [ ]:
# displaying first five rows of the dataframe
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# dropping the target column from the dataframe
df.drop(['target'], axis=1)

,ids,date,flag,user,text
0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [ ]:
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# downloading stop words
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Downloading open multilingual wornet
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
documents = []
stemmer=WordNetLemmatizer()
for i in df['text']:
    document=re.sub(r'\W' , ' ' , str(i))
        #lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    # removing stop words
    document = [word for word in document .split() if word not in stopwords.words("english")]
    document = ' '.join(document)
    
    documents.append(document)
    


In [ ]:
documents

['switchfoot http twitpic com 2y1zl Awww bummer You shoulda got David Carr Third Day D',
 'upset update Facebook texting might cry result School today also Blah',
 'Kenichan I dived many time ball Managed save 50 The rest go bound',
 'whole body feel itchy like fire',
 'nationwideclass behaving mad I see',
 'Kwesidei whole crew',
 'Need hug',
 'LOLTrish hey long time see Yes Rains bit bit LOL I fine thanks',
 'Tatiana_K nope',
 'twittera que muera',
 'spring break plain city snowing',
 'I pierced ear',
 'caregiving I bear watch And I thought UA loss wa embarrassing',
 'octolinz16 It count idk I either never talk anymore',
 'smarrison would first gun really though zac snyder doucheclown',
 'iamjazzyfizzle I wish I got watch I miss iamlilnicki wa premiere',
 'Hollis death scene hurt severely watch film wry director cut',
 'file tax',
 'LettyA ahh ive always wanted see rent love soundtrack',
 'FakerPattyPattz Oh dear Were drinking forgotten table drink',
 'alydesigns wa day get much done'

In [ ]:
clean_data=pd.DataFrame(documents)

In [ ]:
clean_data=clean_data.rename(columns={0:'Content'})
clean_data.head(10)

,Content
0,switchfoot http twitpic com 2y1zl Awww bummer ...
1,upset update Facebook texting might cry result...
2,Kenichan I dived many time ball Managed save 5...
3,whole body feel itchy like fire
4,nationwideclass behaving mad I see
5,Kwesidei whole crew
6,Need hug
7,LOLTrish hey long time see Yes Rains bit bit L...
8,Tatiana_K nope
9,twittera que muera


In [ ]:
pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from textblob import TextBlob

In [ ]:
TextBlob(clean_data["Content"].iloc[0]).sentiment

Sentiment(polarity=0.2, subjectivity=0.45)

In [ ]:
tfidfconverter = TfidfVectorizer(max_features=1000,min_df = 0.005, max_df = 0.98,ngram_range=(1,3),stop_words=stopwords.words('english'))

In [ ]:
converted=tfidfconverter.fit_transform(clean_data['Content'])
converted.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
tfidfconverter.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['10',
 '10 another',
 '10 another day',
 '12',
 '12 gym',
 '12 gym class',
 '12 work',
 '12 work 12',
 '2y1zl',
 '2y1zl awww',
 '2y1zl awww bummer',
 '50',
 '50 rest',
 '50 rest go',
 'account',
 'account aim',
 'account aim make',
 'afternoon',
 'ahh',
 'ahh ive',
 'ahh ive always',
 'aim',
 'aim make',
 'aim make new',
 'alielayus',
 'alielayus want',
 'alielayus want go',
 'almost',
 'almost lover',
 'almost lover exception',
 'also',
 'also blah',
 'also mean',
 'also mean le',
 'always',
 'always wanted',
 'always wanted see',
 'alydesigns',
 'alydesigns wa',
 'alydesigns wa day',
 'anaheim',
 'anaheim may',
 'anaheim may though',
 'angry_barista',
 'angry_barista baked',
 'angry_barista baked cake',
 'another',
 'another day',
 'another day gonna',
 'anymore',
 'asian',
 'asian eye',
 'asian eye sleep',
 'asked',
 'asked meet',
 'asked meet mid',
 'asleep',
 'asleep heard',
 'asleep heard tracy',
 'ated',
 'awe',
 'awe love',
 'awe love miss',
 'awww',
 'awww bummer',
 'awww bum

In [ ]:
len(tfidfconverter.get_feature_names())

904

In [ ]:
clean_data['Content'][:]

0     switchfoot http twitpic com 2y1zl Awww bummer ...
1     upset update Facebook texting might cry result...
2     Kenichan I dived many time ball Managed save 5...
3                       whole body feel itchy like fire
4                    nationwideclass behaving mad I see
5                                   Kwesidei whole crew
6                                              Need hug
7     LOLTrish hey long time see Yes Rains bit bit L...
8                                        Tatiana_K nope
9                                    twittera que muera
10                      spring break plain city snowing
11                                        I pierced ear
12    caregiving I bear watch And I thought UA loss ...
13    octolinz16 It count idk I either never talk an...
14    smarrison would first gun really though zac sn...
15    iamjazzyfizzle I wish I got watch I miss iamli...
16    Hollis death scene hurt severely watch film wr...
17                                             f

In [ ]:
fdist=FreqDist()

In [ ]:
for x in str(clean_data['Content'][:]).split():
    fdist[x]+=1
len(fdist)

315

In [ ]:
fdist.most_common(30)

[('I', 27),
 ('...', 6),
 ('time', 4),
 ('see', 4),
 ('wa', 4),
 ('sad', 4),
 ('1', 3),
 ('cry', 3),
 ('3', 3),
 ('8', 3),
 ('12', 3),
 ('watch', 3),
 ('get', 3),
 ('one', 3),
 ('Just', 3),
 ('http', 2),
 ('Awww', 2),
 ('whole', 2),
 ('body', 2),
 ('feel', 2),
 ('like', 2),
 ('mad', 2),
 ('5', 2),
 ('6', 2),
 ('bit', 2),
 ('break', 2),
 ('thought', 2),
 ('either', 2),
 ('really', 2),
 ('wish', 2)]

In [ ]:
clean_data['Polarity']=clean_data['Content'].apply(lambda x: TextBlob(x).sentiment[0])
clean_data['Subjectivity']=clean_data['Content'].apply(lambda x: TextBlob(x).sentiment[1])
clean_data['Length']=clean_data['Content'].apply(lambda x: len(x.split()))
clean_data.head(20)

,Content,Polarity,Subjectivity,Length
0,switchfoot http twitpic com 2y1zl Awww bummer ...,0.200000,0.450000,15
1,upset update Facebook texting might cry result...,0.000000,0.000000,11
2,Kenichan I dived many time ball Managed save 5...,0.500000,0.500000,13
3,whole body feel itchy like fire,0.200000,0.400000,6
4,nationwideclass behaving mad I see,-0.625000,1.000000,5
5,Kwesidei whole crew,0.200000,0.400000,3
6,Need hug,0.000000,0.000000,2
7,LOLTrish hey long time see Yes Rains bit bit L...,0.341667,0.450000,13
8,Tatiana_K nope,0.000000,0.000000,2
9,twittera que muera,0.000000,0.000000,3


In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
import nltk.sentiment.vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
analyzer=SentimentIntensityAnalyzer()
scores=[]
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
for i in range(clean_data['Content'].shape[0]):
      compound =  analyzer.polarity_scores(clean_data['Content'][i])["compound"]
      pos =  analyzer.polarity_scores(clean_data['Content'][i])["pos"]
      neu =  analyzer.polarity_scores(clean_data['Content'][i])["neu"]
      neg =  analyzer.polarity_scores(clean_data['Content'][i])["neg"]
      scores.append({"Compound":compound,
                     "Positve":pos,
                     "Negative":neg,
                     "Neutral":neu
                    })
sentiments_score=pd.DataFrame.from_dict(scores)
df=clean_data.join(sentiments_score,how='outer')
df.head(40)

,Content,Polarity,Subjectivity,Length,Compound,Positve,Negative,Neutral
0,switchfoot http twitpic com 2y1zl Awww bummer ...,0.200000,0.450000,15,-0.3818,0.000,0.167,0.833
1,upset update Facebook texting might cry result...,0.000000,0.000000,11,-0.7269,0.000,0.470,0.530
2,Kenichan I dived many time ball Managed save 5...,0.500000,0.500000,13,0.4939,0.225,0.000,0.775
3,whole body feel itchy like fire,0.200000,0.400000,6,-0.2500,0.250,0.450,0.300
4,nationwideclass behaving mad I see,-0.625000,1.000000,5,-0.4939,0.000,0.516,0.484
5,Kwesidei whole crew,0.200000,0.400000,3,0.0000,0.000,0.000,1.000
6,Need hug,0.000000,0.000000,2,0.4767,0.756,0.000,0.244
7,LOLTrish hey long time see Yes Rains bit bit L...,0.341667,0.450000,13,0.8730,0.577,0.000,0.423
8,Tatiana_K nope,0.000000,0.000000,2,0.0000,0.000,0.000,1.000
9,twittera que muera,0.000000,0.000000,3,0.0000,0.000,0.000,1.000
